# Project: Investigate the FBI Gun Data

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
    <ul>
    <li><a href="#gun_explore">Gun Data Exploration</a></li>
    <li><a href="#census_explore">Census Data Exploration</a></li>
    </ul>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

Two datasets are taken into consideration for this project; the _FBI's National Instant Criminal Background Check System_ and the _U.S. Census Data_.

#### The the FBI's National Instant Criminal Background Check System 

This ([here](https://www.google.com/url?q=https://d17h27t6h515a5.cloudfront.net/topher/2017/November/5a0a4db8_gun-data/gun-data.xlsx&sa=D&ust=1532469042127000)) database was provided as an .xlsx file and it has been converted to .csv from Excel. Therefore the starting data is found in the file **gun_data.csv**.
It goes from Novembre 1998 to September 2017, collected monthly (data up to 2020 are available, but the census data here would not cover that period)

Rows: US States

Columns:
 - checks data columns show the different reasons a shop owner can request a check. Every time gun shops are requested a transaction, they need to fill a form ([this](https://www.fbi.gov/services/cjis/nics)). This may result in
   - a permit check or recheck (if the buyer already owns a permit)
   - or in a background check. The background check is subdivided by
     - the type of arm: handgun, long gun, other, multiple or admin
     - the purpose: new buy (not written in the column title), pre-pawn, redemption, returned/disposition, rentals, private sale, return to seller 
 - The first two columns are for the month and the State the data are about
 - Last column is the total number of transactions/checks initiated. __Not neccessarily they have been concluded__.

NB: a lot of transactions are concluded privately or illegally and therefore there is no record about.
 

#### The _U.S. Census Data 

This database ([here](https://www.google.com/url?q=https://d17h27t6h515a5.cloudfront.net/topher/2017/November/5a0a554c_u.s.-census-data/u.s.-census-data.csv&sa=D&ust=1532469042128000)), which was already in .csv format, had the name changed in **us_census_data.csv** to be more Python friendly.
 - This dataset contains information from the 2010 census and estimates of its changes up to 2016 ([here](https://www.census.gov/quickfacts/fact/note/US/PST120218) some explanation)
 - each column represents a US State, but the first -"Fact"- and second -"Fact note"-
 - Each rows a "Fact" (e.g.: "Population, Census, April 1, 2010", "Population, percent change - April 1, 2010 (estimates base) to July 1, 2016")

In [1]:
# import statements

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

<a id='gun_explore'></a>
#### Gun data exploration

In [2]:
# Load data and print out a few lines

df_gun = pd.read_csv('gun_data.csv')
df_gun.head()

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2017-09,Alabama,16717.0,0.0,5734.0,6320.0,221.0,317,0.0,15.0,...,0.0,0.0,0.0,9.0,16.0,3.0,0.0,0.0,3.0,32019
1,2017-09,Alaska,209.0,2.0,2320.0,2930.0,219.0,160,0.0,5.0,...,0.0,0.0,0.0,17.0,24.0,1.0,0.0,0.0,0.0,6303
2,2017-09,Arizona,5069.0,382.0,11063.0,7946.0,920.0,631,0.0,13.0,...,0.0,0.0,0.0,38.0,12.0,2.0,0.0,0.0,0.0,28394
3,2017-09,Arkansas,2935.0,632.0,4347.0,6063.0,165.0,366,51.0,12.0,...,0.0,0.0,0.0,13.0,23.0,0.0,0.0,2.0,1.0,17747
4,2017-09,California,57839.0,0.0,37165.0,24581.0,2984.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123506


In [3]:
# a little general exploration
df_gun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12485 entries, 0 to 12484
Data columns (total 27 columns):
month                        12485 non-null object
state                        12485 non-null object
permit                       12461 non-null float64
permit_recheck               1100 non-null float64
handgun                      12465 non-null float64
long_gun                     12466 non-null float64
other                        5500 non-null float64
multiple                     12485 non-null int64
admin                        12462 non-null float64
prepawn_handgun              10542 non-null float64
prepawn_long_gun             10540 non-null float64
prepawn_other                5115 non-null float64
redemption_handgun           10545 non-null float64
redemption_long_gun          10544 non-null float64
redemption_other             5115 non-null float64
returned_handgun             2200 non-null float64
returned_long_gun            2145 non-null float64
returned_other   

In [4]:
# what are the States and jurisdictions taken into consideration
states = df_gun['state'].unique()
print(states)
print(len(states))

['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Guam' 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Mariana Islands' 'Maryland' 'Massachusetts'
 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana' 'Nebraska'
 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'Puerto Rico' 'Rhode Island' 'South Carolina' 'South Dakota' 'Tennessee'
 'Texas' 'Utah' 'Vermont' 'Virgin Islands' 'Virginia' 'Washington'
 'West Virginia' 'Wisconsin' 'Wyoming']
55


##### General
There are 12485 entries and 27 columns.

0.0 are seen as nulls.

3 columns don't contain nulls: _month, state_ and _totals_.

The database contains information about 55 US States and other jurisdictions.

##### Data
_Month_ and _state_ are string.
All other columns, but _multiple_ and _totals_ are floats.
_Multiple_ and _totals_ are integers.

- _State_ is OK as string,
- _month_ could be transformed in datetime,
- all others could be integers.

<a id='census_explore'></a>
#### Census Data exploration

In [5]:
# import census data
df_census = pd.read_csv('us_census_data.csv')
df_census.head(2)

,Fact,Fact Note,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,"Population estimates, July 1, 2016, (V2016)",NaN,"4,863,300","741,894","6,931,071","2,988,248","39,250,017","5,540,545","3,576,452","952,065",...,865454,6651194,"27,862,596","3,051,217","624,594","8,411,808","7,288,000","1,831,102","5,778,708","585,501"
1,"Population estimates base, April 1, 2010, (V2...",NaN,"4,780,131","710,249","6,392,301","2,916,025","37,254,522","5,029,324","3,574,114","897,936",...,814195,6346298,"25,146,100","2,763,888","625,741","8,001,041","6,724,545","1,853,011","5,687,289","563,767"


In [6]:
# general info
df_census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 52 columns):
Fact              80 non-null object
Fact Note         28 non-null object
Alabama           65 non-null object
Alaska            65 non-null object
Arizona           65 non-null object
Arkansas          65 non-null object
California        65 non-null object
Colorado          65 non-null object
Connecticut       65 non-null object
Delaware          65 non-null object
Florida           65 non-null object
Georgia           65 non-null object
Hawaii            65 non-null object
Idaho             65 non-null object
Illinois          65 non-null object
Indiana           65 non-null object
Iowa              65 non-null object
Kansas            65 non-null object
Kentucky          65 non-null object
Louisiana         65 non-null object
Maine             65 non-null object
Maryland          65 non-null object
Massachusetts     65 non-null object
Michigan          65 non-null object
Minnesota

##### General
There are 85 rows and 52 columns.

The first 2 columns are "Fact" and "Fact notes", therefore only the 50 US States are considered (no other jurisdictions). __This list differs from the df_gun one.__

##### Data
There are spaces in column names that can be replaced by underscores.

All cells contain strings. Excluding the first 2 columns, everything will be better as float. __NB: here the instances are the columns, data differ for each row -the "fact"-, so different rows contain different kind of data (e.g. units, percentages, amounts in dollars)__ 

There are a lot of Nulls.

__Before dealing with Nulls and data types is better to select the rows we are interested in__

### Data Cleaning

#### Gun data
- Adjust datatypes

In [7]:
# change month from string to datetime (it adds the day, but it is no problem)
df_gun['month'] = pd.to_datetime(df_gun['month'])

# change other columns to int
df_gun.fillna(0, inplace=True) # replace nulls, otherwise astype won't work
df_gun.iloc[:,2:26] = df_gun.iloc[:,2:26].astype('int')


In [8]:
df_gun.head()

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2017-09-01,Alabama,16717,0,5734,6320,221,317,0,15,...,0,0,0,9,16,3,0,0,3,32019
1,2017-09-01,Alaska,209,2,2320,2930,219,160,0,5,...,0,0,0,17,24,1,0,0,0,6303
2,2017-09-01,Arizona,5069,382,11063,7946,920,631,0,13,...,0,0,0,38,12,2,0,0,0,28394
3,2017-09-01,Arkansas,2935,632,4347,6063,165,366,51,12,...,0,0,0,13,23,0,0,2,1,17747
4,2017-09-01,California,57839,0,37165,24581,2984,0,0,0,...,0,0,0,0,0,0,0,0,0,123506


#### Census data
- Eliminate spaces in column names
- Adjust datatypes
- Deal with NaNs

In [9]:
# Eliminate spaces in column names and make everithing lowercase
df_census.rename(columns=lambda x : x.lower().replace(" ", "_"), inplace=True)

In [10]:
df_census.head(1)

,fact,fact_note,alabama,alaska,arizona,arkansas,california,colorado,connecticut,delaware,...,south_dakota,tennessee,texas,utah,vermont,virginia,washington,west_virginia,wisconsin,wyoming
0,"Population estimates, July 1, 2016, (V2016)",NaN,"4,863,300","741,894","6,931,071","2,988,248","39,250,017","5,540,545","3,576,452","952,065",...,865454,6651194,"27,862,596","3,051,217","624,594","8,411,808","7,288,000","1,831,102","5,778,708","585,501"


##### Look for columns and rows we are interested in

A lot of rows in the Census data are empty, but in the first 2 columns: it is were the notes have been put in. There are more practical to have a look at in the original Excel file, but we are not interested because they are relative to rows we won't keep.

Then we will have a look at the facts the rows are about and keep only the ones we want to work with.

Later we are going to do the same on the FBI Gun data and, last, we will compare the list of States, to keep only the overlapping ones.

In [12]:
# drop rows completely empty (excluded data in the first 2 columns)
df_census = df_census.dropna(subset=df_census.columns[2:])

In [20]:
print(df_census.fact.to_string(index=False))

      Population estimates, July 1, 2016,  (V2016)
 Population estimates base, April 1, 2010,  (V2...
 Population, percent change - April 1, 2010 (es...
                 Population, Census, April 1, 2010
 Persons under 5 years, percent, July 1, 2016, ...
     Persons under 5 years, percent, April 1, 2010
 Persons under 18 years, percent, July 1, 2016,...
    Persons under 18 years, percent, April 1, 2010
 Persons 65 years and over, percent,  July 1, 2...
 Persons 65 years and over, percent, April 1, 2010
  Female persons, percent,  July 1, 2016,  (V2016)
            Female persons, percent, April 1, 2010
      White alone, percent, July 1, 2016,  (V2016)
 Black or African American alone, percent, July...
 American Indian and Alaska Native alone, perce...
      Asian alone, percent, July 1, 2016,  (V2016)
 Native Hawaiian and Other Pacific Islander alo...
 Two or More Races, percent, July 1, 2016,  (V2...
 Hispanic or Latino, percent, July 1, 2016,  (V...
 White alone, not Hispanic or L

In [ ]:
# we'll keep rows about population in 2010 and 2016 estimates, income in 2011-2015, land area in square miles
rows_to_keep = df_census.iloc[[0, 1, 6, 7, 8, 9, 34, 35, 47, 48, 63], :

In [ ]:
# astype doesn't like numbers with commas (e.g. 4,863,300)
#for c in df_census.columns[2:]:
#    df_census[c] = df_census[c].str.replace("%","")
#    df_census[c] = df_census[c].str.replace(",","")
#    df_census[c] = df_census[c].str.replace("$","")

#df_census.loc[df_census['connecticut'] == '"01"']

# convert strings into floats
#df_census.iloc[:,2:].astype(float)

Index(['alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado',
       'connecticut', 'delaware', 'florida', 'georgia', 'hawaii', 'idaho',
       'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana',
       'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota',
       'mississippi', 'missouri', 'montana', 'nebraska', 'nevada',
       'new_hampshire', 'new_jersey', 'new_mexico', 'new_york',
       'north_carolina', 'north_dakota', 'ohio', 'oklahoma', 'oregon',
       'pennsylvania', 'rhode_island', 'south_carolina', 'south_dakota',
       'tennessee', 'texas', 'utah', 'vermont', 'virginia', 'washington',
       'west_virginia', 'wisconsin', 'wyoming'],
      dtype='object')

In [48]:
#fact_list = df_census['Fact']
#print(df_census.Fact.to_string(index=False))
#df_census.iloc[:,30:40].tail(5)

,new_hampshire,new_jersey,new_mexico,new_york,north_carolina,north_dakota,ohio,oklahoma,oregon,pennsylvania
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# drop rows completely empty (excluded the first 2 columns)
df_census.dropna(subset=df_census.columns[2:])

,fact,fact_note,alabama,alaska,arizona,arkansas,california,colorado,connecticut,delaware,...,south_dakota,tennessee,texas,utah,vermont,virginia,washington,west_virginia,wisconsin,wyoming
0,"Population estimates, July 1, 2016, (V2016)",NaN,"4,863,300","741,894","6,931,071","2,988,248","39,250,017","5,540,545","3,576,452","952,065",...,865454,6651194,"27,862,596","3,051,217","624,594","8,411,808","7,288,000","1,831,102","5,778,708","585,501"
1,"Population estimates base, April 1, 2010, (V2...",NaN,"4,780,131","710,249","6,392,301","2,916,025","37,254,522","5,029,324","3,574,114","897,936",...,814195,6346298,"25,146,100","2,763,888","625,741","8,001,041","6,724,545","1,853,011","5,687,289","563,767"
2,"Population, percent change - April 1, 2010 (es...",NaN,1.70%,4.50%,8.40%,2.50%,5.40%,10.20%,0.10%,6.00%,...,0.063,0.048,10.80%,10.40%,-0.20%,5.10%,8.40%,-1.20%,1.60%,3.90%
3,"Population, Census, April 1, 2010",NaN,"4,779,736","710,231","6,392,017","2,915,918","37,253,956","5,029,196","3,574,097","897,934",...,814180,6346105,"25,145,561","2,763,885","625,741","8,001,024","6,724,540","1,852,994","5,686,986","563,626"
4,"Persons under 5 years, percent, July 1, 2016, ...",NaN,6.00%,7.30%,6.30%,6.40%,6.30%,6.10%,5.20%,5.80%,...,0.071,0.061,7.20%,8.30%,4.90%,6.10%,6.20%,5.50%,5.80%,6.50%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,"Veteran-owned firms, 2012",NaN,"41,943","7,953","46,780","25,915","252,377","51,722","31,056","7,206",...,8604,59379,"213,590","18,754","8,237","76,434","49,331","12,912","39,830","6,470"
61,"Nonveteran-owned firms, 2012",NaN,"316,984","56,091","427,582","192,988","3,176,341","469,524","281,182","60,318",...,66219,469392,"2,057,218","219,807","63,317","548,439","461,401","94,960","370,755","51,353"
62,"Population per square mile, 2010",NaN,94.4,1.2,56.3,56,239.1,48.5,738.1,460.8,...,10.7,153.9,96.3,33.6,67.9,202.6,101.2,77.1,105,5.8
63,"Land area in square miles, 2010",NaN,"50,645.33","570,640.95","113,594.08","52,035.48","155,779.22","103,641.89","4,842.36","1,948.54",...,75811,41234.9,"261,231.71","82,169.62","9,216.66","39,490.09","66,455.52","24,038.21","54,157.80","97,093.14"


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!